In [1]:
import pandas as pd
import numpy as np

In [2]:
##Final code for USA Counties, exporting to CSV to use in the .py file with Dash

In [3]:
countiesDF = pd.read_csv("Resources/countypres_2000-2016.csv")

In [4]:
df2016 = countiesDF[(countiesDF["year"] == 2016)]
df2016_reps = df2016[(df2016["party"]) == "republican"]
df2016_reps["percent"] = df2016_reps["candidatevotes"] / df2016["totalvotes"]

df2016_reps.dtypes

/Users/mak/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


year                int64
state              object
state_po           object
county             object
FIPS              float64
office             object
candidate          object
party              object
candidatevotes    float64
totalvotes          int64
version             int64
percent           float64
dtype: object

In [5]:
df2016_reps = df2016_reps.dropna(subset = ["FIPS"])
df2016_reps["FIPS"] = df2016_reps["FIPS"].astype(np.int64)
df2016_reps["FIPS"] = df2016_reps["FIPS"].astype(str)

In [6]:
i = 0
while i < len(df2016_reps.index):

    if len(df2016_reps.iloc[i,4])<=4:
        df2016_reps.iloc[i,4] = "0" + df2016_reps.iloc[i,4]
        i+=1
    else:
        i+=1

df2016_reps.head(10)

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version,percent
40518,2016,Alabama,AL,Autauga,01001,President,Donald Trump,republican,18172.0,24973,20191203,0.727666
40521,2016,Alabama,AL,Baldwin,01003,President,Donald Trump,republican,72883.0,95215,20191203,0.765457
40524,2016,Alabama,AL,Barbour,01005,President,Donald Trump,republican,5454.0,10469,20191203,0.520967
40527,2016,Alabama,AL,Bibb,01007,President,Donald Trump,republican,6738.0,8819,20191203,0.764032
40530,2016,Alabama,AL,Blount,01009,President,Donald Trump,republican,22859.0,25588,20191203,0.893348
40533,2016,Alabama,AL,Bullock,01011,President,Donald Trump,republican,1140.0,4710,20191203,0.242038
40536,2016,Alabama,AL,Butler,01013,President,Donald Trump,republican,4901.0,8732,20191203,0.561269
40539,2016,Alabama,AL,Calhoun,01015,President,Donald Trump,republican,32865.0,47864,20191203,0.686633
40542,2016,Alabama,AL,Chambers,01017,President,Donald Trump,republican,7843.0,13900,20191203,0.564245
40545,2016,Alabama,AL,Cherokee,01019,President,Donald Trump,republican,8953.0,10733,20191203,0.834156


In [7]:
def label_pct(row):
    if row["percent"] <= .40:
        return 4

    if row["percent"] <= .60:
        return 6

    return 10


In [8]:
def pop_rank(row):
    return pd.qcut(row,5, retbins = True)


In [9]:
def combined_rank(row):
    return row["pop_rank"]*100+row["pct_rank"]

In [10]:
df2016_reps["pct_rank"] = df2016_reps.apply (lambda row: label_pct(row), axis=1)
df2016_reps["pop_rank"] = pd.qcut(df2016_reps["totalvotes"], q=[0, .25, .5, .75, 1],labels =[1,2,3,4] )
df2016_reps["combined_rank"] = df2016_reps.apply (lambda row: combined_rank(row), axis=1)

df2016_reps["combined_rank"] = df2016_reps["combined_rank"].astype(str)

In [11]:
df2016_reps.to_csv("Resources/usa_counties.csv", index = False)

In [12]:
##Final code for MN Precincts, exporting to CSV to use in the .py file with Dash

In [13]:
csvpath = "Resources/2016_precincts_newVersion.csv"
prec_df = pd.read_csv(csvpath, encoding="ISO-8859-1")

In [14]:
prec_df = prec_df[['VTDID', 'PCTNAME', 'PCTCODE', 'MCDNAME', 'COUNTYNAME', 'COUNTYCODE',
                   'USPRSR', 'USPRSDFL', 'USPRSTOTAL']][:-1]

In [15]:
prec_df = prec_df.rename(columns = {'MCDNAME': 'Munic/Unorg Terr Name',
                                    'USPRSR': 'Trump', 'USPRSDFL': 'Clinton',
                                    'USPRSTOTAL': 'Total Votes'})

In [16]:
prec_df['percent'] = prec_df['Trump'] / prec_df['Total Votes']

In [17]:
prec_df["pct_rank"] = prec_df.apply (lambda row: label_pct(row), axis=1)
prec_df["pop_rank"] = pd.qcut(prec_df["Total Votes"], q=[0, .25, .5, .75, 1],labels =[1,2,3,4] )
prec_df["combined_rank"] = prec_df.apply (lambda row: combined_rank(row), axis=1)
prec_df["combined_rank"] = prec_df["combined_rank"].astype(str)

In [18]:
prec_df.to_csv("Resources/mn_precincts.csv", index = False)